In [6]:
import pandas as pd
import networkx as nx
import csv
import matplotlib.pyplot as plt
from collections import Counter

In [9]:
# Read the CSV file and extract relevant data
def read_transactions(file_path):
    transactions = []
    with open(file_path, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            transactions.append({
                'from': row['from'],
                'to': row['to'],
                'value': row['value'],
                'hash': row['hash'],
                'timeStamp': row['timeStamp'],
                'blockNumber': row['blockNumber']
            })
    return transactions

# Create a directed graph from the transactions
def create_graph(transactions):
    G = nx.DiGraph()  # Directed graph since transactions are one-way (from -> to)
    
    for tx in transactions:
        from_addr = tx['from']
        to_addr = tx['to']
        value = float(tx['value'])
        
        # Add nodes (wallet addresses)
        G.add_node(from_addr)
        G.add_node(to_addr)
        
        # Add an edge for each transaction
        G.add_edge(from_addr, to_addr, weight=value)
    
    return G

In [10]:
from sklearn.cluster import KMeans
import numpy as np

# Function to extract features from the graph for each wallet
def extract_features(G):
    features = []
    addresses = []
    
    for node in G.nodes():
        in_degree = G.in_degree(node)
        out_degree = G.out_degree(node)
        total_out_value = sum([G[u][v]['weight'] for u, v in G.out_edges(node)])  # Sum of outgoing transactions
        total_in_value = sum([G[u][v]['weight'] for u, v in G.in_edges(node)])  # Sum of incoming transactions
        
        # Append features: in-degree, out-degree, total value sent, total value received
        features.append([in_degree, out_degree, total_in_value, total_out_value])
        addresses.append(node)
    
    return np.array(features), addresses

# Function to run K-Means on the extracted features
def run_kmeans(features, addresses, n_clusters=3):
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(features)
    labels = kmeans.labels_
    
    # Group addresses by cluster
    clusters = {}
    for i, label in enumerate(labels):
        if label not in clusters:
            clusters[label] = []
        clusters[label].append(addresses[i])
    
    return clusters

# Main function to run K-Means clustering
def main_kmeans(file_path):
    # Step 1: Read the transactions from CSV
    transactions = read_transactions(file_path)
    
    # Step 2: Create the graph
    G = create_graph(transactions)
    
    # Step 3: Extract features from the graph
    features, addresses = extract_features(G)
    
    # Step 4: Run K-Means clustering
    clusters = run_kmeans(features, addresses, n_clusters=3)  # You can adjust the number of clusters
    
    # Print the addresses in each cluster
    for cluster_id, cluster_addresses in clusters.items():
        print(f"Cluster {cluster_id}: {cluster_addresses}")

# Example usage
file_path = 'ethersccan_2000.csv'  # Path to your CSV file
main_kmeans(file_path)


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Cluster 0: ['...']
Cluster 2: ['0xdfd5293d8e347dfe59e90efd55b2956a1343963d', '0x28c6c06298d514db089934071355e5743bf21d60', '0x9696f59e4d72e237be84ffd425dcad154bf96976']
Cluster 1: ['0x7bea16f5553eb3dadb23860abd29c55dd891ff70']


In [11]:
file_path = 'ethersccan.csv'  # Path to your CSV file
main_kmeans(file_path)

Cluster 0: ['...']
Cluster 2: ['0x21a31ee1afc51d94c2efccaa2092ad1028285549', '0x3a82ecee4f915ffc0a8a221e3c0cb1db403a3113', '0xaddda8bba4043b935b066b5a8b462c589ca8fc0a', '0x533caaeaee0396bed2714c547104591ca3e2f345', '0x99cbb867ff038057dda7b78e5f7e5beadbe3bebb', '0x1757d009d24c87959ea88c24cbfba40088da335e', '0xb72301e6f910cbcb474749e7b77f6ee41c259123']
Cluster 1: ['0x1851cbb368c7c49b997064086da94dbad90eb9b5']


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [12]:
import csv
import numpy as np
from sklearn.cluster import DBSCAN
import networkx as nx
import matplotlib.pyplot as plt

# Read the CSV file and extract relevant data
def read_transactions(file_path):
    transactions = []
    with open(file_path, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            transactions.append({
                'from': row['from'],
                'to': row['to'],
                'value': float(row['value']),
                'hash': row['hash'],
                'timeStamp': row['timeStamp'],
                'blockNumber': row['blockNumber']
            })
    return transactions

# Create a directed graph from the transactions
def create_graph(transactions):
    G = nx.DiGraph()  # Directed graph since transactions are one-way (from -> to)
    
    for tx in transactions:
        from_addr = tx['from']
        to_addr = tx['to']
        value = tx['value']
        
        # Add nodes (wallet addresses)
        G.add_node(from_addr)
        G.add_node(to_addr)
        
        # Add an edge for each transaction
        G.add_edge(from_addr, to_addr, weight=value)
    
    return G

# Generate features for each address
def generate_features(G):
    features = []
    addresses = list(G.nodes())
    
    for address in addresses:
        outgoing_tx = G.out_degree(address)
        incoming_tx = G.in_degree(address)
        total_out_value = sum([G.get_edge_data(address, nbr).get('weight', 0) for nbr in G.successors(address)])
        total_in_value = sum([G.get_edge_data(nbr, address).get('weight', 0) for nbr in G.predecessors(address)])
        unique_counterparties = len(set(G.successors(address)) | set(G.predecessors(address)))
        
        # Create a feature vector for each address
        features.append([outgoing_tx, incoming_tx, total_out_value, total_in_value, unique_counterparties])
    
    return np.array(features), addresses

# Apply DBSCAN to detect Sybil attacks
def detect_sybil_dbscan(features, addresses, eps=0.5, min_samples=2):
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(features)
    
    # Identify outliers (label == -1)
    flagged_addresses = [addresses[i] for i, label in enumerate(labels) if label == -1]
    
    return flagged_addresses

# Main function
def main(file_path):
    # Step 1: Read the transactions from CSV
    transactions = read_transactions(file_path)
    
    # Step 2: Create the graph
    G = create_graph(transactions)
    
    # Step 3: Generate features for DBSCAN
    features, addresses = generate_features(G)
    
    # Step 4: Apply DBSCAN to detect Sybil attacks
    flagged_addresses = detect_sybil_dbscan(features, addresses)
    
    # Step 5: Output the flagged addresses
    print(f"Flagged addresses (potential Sybil attackers): {len(flagged_addresses)}")
    unique_addresses = set(flagged_addresses)
    print(f"Unique addresses: {len(unique_addresses)}")


main('ethersccan.csv')


Flagged addresses (potential Sybil attackers): 1117
Unique addresses: 1117
